In [7]:
from spektral import datasets

# Load the citation data.
cora = datasets.citation.Citation("cora").read()[0]

In [22]:
import tensorflow as tf

# Convert citation graph into an undirected graph.
adjacency = cora.a.todense()
adjacency_upper = tf.linalg.band_part(adjacency, 0, -1)
adjacency_lower = tf.linalg.band_part(adjacency, -1, 0)

adjacency_upper_symmetric = adjacency_upper + tf.transpose(adjacency_upper)
adjacency_lower_symmetric = adjacency_lower + tf.transpose(adjacency_lower)
adjacency_undirected = tf.maximum(adjacency_upper_symmetric, adjacency_lower_symmetric)

<tf.Tensor: shape=(2708, 2708), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)>

In [29]:
from cotton_flower_mot.pipelines.model_training.graph_utils import compute_pairwise_similarities
from cotton_flower_mot.pipelines.model_training.similarity_utils import cosine_similarity

# Compute edge features.
connected_node_indices = tf.where(adjacency)
# Get the corresponding node features for each edge.
left_node_features = tf.gather(cora.x, connected_node_indices[:, 0])
right_node_features = tf.gather(cora.x, connected_node_indices[:, 1])
# Compute cosine similarities for each edge.
cosine_similarities = cosine_similarity(left_node_features, right_node_features)

# Compute edge direction vectors.
edge_directions = tf.stack([adjacency_upper_symmetric, adjacency_lower_symmetric], axis=-1)
edge_directions = tf.reshape(edge_directions, (-1, 2))
tf.reduce_all(adjacency == adjacency_undirected)

<tf.Tensor: shape=(), dtype=bool, numpy=True>